In [ ]:
import pandas as pd

dataFile_emptyOutputs = pd.read_csv('test_data1_WithOutputs.csv')
print(dataFile_emptyOutputs.describe())

In [ ]:
print(dataFile_emptyOutputs.isna().sum())
print(dataFile_emptyOutputs.info())

In [ ]:
import missingno as mno
mno.matrix(dataFile_emptyOutputs)

In [ ]:
missing_columns = ['Torque']
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

In [ ]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

for feature in missing_columns:
    dataFile_emptyOutputs[feature+'_imp'] = dataFile_emptyOutputs[feature]
    dataFile_emptyOutputs = random_imputation(dataFile_emptyOutputs,feature)
print(dataFile_emptyOutputs)

In [ ]:
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])
dataFile_emptyOutputs = dataFile_emptyOutputs.drop(['Sr No','Screw Configuration','Experiments','Liq add position','Regime','Beta','d50','MRT','Exp Fill level'],axis=1)

for feature in missing_columns:
        
    deter_data["Det" + feature] = dataFile_emptyOutputs[feature + "_imp"]
    parameters = list(set(dataFile_emptyOutputs.columns) - set(missing_columns) - {feature + '_imp'})
    print(parameters)
   

In [ ]:
for feature in missing_columns:
 #Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = dataFile_emptyOutputs[parameters], y = dataFile_emptyOutputs[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    deter_data.loc[dataFile_emptyOutputs[feature].isnull(), "Det" + feature] = model.predict(dataFile_emptyOutputs[parameters])[dataFile_emptyOutputs[feature].isnull()]
